<a href="https://colab.research.google.com/github/GraphGauge/ObliqueNet/blob/main/DifferentialPrivacy_DBQueries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
num_entries = 5000
db = torch.rand(num_entries) > 0.5
db

tensor([False,  True,  True,  ...,  True,  True,  True])

Generate Parallel Database

In [4]:
def create_parallel_db(db, remove_index):
    return torch.cat((db[0:remove_index], db[remove_index+1:]))

In [5]:
def create_parallel_dbs(db):
    parallel_dbs = list()
    for i in range(len(db)):
        pdb = create_parallel_db(db, i)
        parallel_dbs.append(pdb)
    return parallel_dbs

In [6]:
def create_db_and_parallels(num_entries):
    # generate dbs and parallel dbs on the fly
    db = torch.rand(num_entries) > 0.5
    pdbs = create_parallel_dbs(db)
    
    return db, pdbs

In [7]:
db, pdbs = create_db_and_parallels(10)
pdbs
print("Real database:", db)
print("Size of real DB", db.size())
print("A sample parallel DB", pdbs[0])
print("Size of parallel DB", pdbs[0].size())

Real database: tensor([False, False, False,  True, False,  True,  True, False, False, False])
Size of real DB torch.Size([10])
A sample parallel DB tensor([False, False,  True, False,  True,  True, False, False, False])
Size of parallel DB torch.Size([9])


In [8]:
db, pdbs = create_db_and_parallels(200)
def query(db):
    return db.sum()

In [9]:
query(db)

tensor(108)

In [10]:
query(pdbs[1])

tensor(107)

In [11]:
full_db_result = query(db)
print(full_db_result)

tensor(108)


In [13]:
sensitivity = 0
sensitivity_scale = []
for pdb in pdbs:
    pdb_result = query(pdb)
    db_distance = torch.abs(pdb_result - full_db_result)
    if(db_distance > sensitivity):
        sensitivity_scale.append(db_distance)
        sensitivity = db_distance

In [14]:
sensitivity

tensor(1)

In [15]:
def sensitivity(query, num_entries=1000):
    db, pdbs = create_db_and_parallels(num_entries)
    
    full_db_result = query(db)
    
    max_distance = 0
    for pdb in pdbs:
        # for each parallel db, execute the query (sum, or mean, ..., etc)
        pdb_result = query(pdb)
        db_distance = torch.abs(pdb_result - full_db_result)
        
        if (db_distance > max_distance):
            max_distance = db_distance

    return max_distance

In [17]:
# our query is now the mean
def query(db):
    return db.float().mean()

In [18]:
sensitivity(query)

tensor(0.0005)